# SPARC Containerization and Deployment

## 1.0 Introduction
This notebook covers packaging SPARC services as portable containers and preparing both local Podman validation and HiPerGator production deployment artifacts.

### 1.1 Objectives
1. **Containerize**: Build images for MAS backend, Unity Linux server runtime, and Signaling Server.
2. **Orchestrate**: Validate local Podman pod networking for server-side rendering (Pixel Streaming).
3. **Deploy**: Generate production SLURM artifacts for HiPerGator-compatible backend workflows.

### 1.2 Introduction Diagram
![Introduction](./images/notebook_2_-_section_1.png)

Introduction: This section defines the containerization objectives for the SPARC stack. We now support server-side rendering for thin clients by introducing a Unity Linux server runtime and a signaling container in addition to the backend services.

## 2.0 Containerization (Docker/Podman -> Apptainer)
We develop with Docker/Podman and deploy with Apptainer on HPC when needed.

![notebook 2 - section 2.png](images/notebook_2_-_section_2.png)

Container Build Strategy: The flow uses secure, minimal runtime images. Build steps compile dependencies in dedicated stages, then copy only required artifacts into runtime images.

This notebook provides image definitions for three build targets:
1. **MAS Backend** (`Dockerfile.mas`)
2. **Unity Linux Server Build** (`Dockerfile.unity-server`)
3. **WebRTC Signaling Server** (`Dockerfile.signaling`)

### 2.1 Dockerfile Definition

This section includes reference Dockerfiles for the MAS backend, Unity Linux server rendering runtime, and the Unity Render Streaming signaling server.

The Unity runtime image should use an NVIDIA OpenGL runtime base and launch the Linux player with Render Streaming-compatible flags.
The signaling image should be a lightweight Node.js runtime built from the Unity Render Streaming signaling package.

In [ ]:
# 2.2-2.4 Dockerfiles for MAS, Unity Linux Server, and Signaling
def create_dockerfiles():
    mas = """
# --- Build Stage ---
FROM python:3.11-slim as builder
WORKDIR /app
RUN apt-get update && apt-get install -y --no-install-recommends \",
,
,